# Fetch Analytics Engineer - Coding Excercise

In [1]:
# import packages
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy import text

In [2]:
# read json files
receipts_df = pd.read_json('receipts.json', lines = True)
users_df = pd.read_json('users.json',  lines = True)
brands_df = pd.read_json('brands.json',  lines = True)

In [3]:
receipts_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1119 entries, 0 to 1118
Data columns (total 15 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   _id                      1119 non-null   object 
 1   bonusPointsEarned        544 non-null    float64
 2   bonusPointsEarnedReason  544 non-null    object 
 3   createDate               1119 non-null   object 
 4   dateScanned              1119 non-null   object 
 5   finishedDate             568 non-null    object 
 6   modifyDate               1119 non-null   object 
 7   pointsAwardedDate        537 non-null    object 
 8   pointsEarned             609 non-null    float64
 9   purchaseDate             671 non-null    object 
 10  purchasedItemCount       635 non-null    float64
 11  rewardsReceiptItemList   679 non-null    object 
 12  rewardsReceiptStatus     1119 non-null   object 
 13  totalSpent               684 non-null    float64
 14  userId                  

In [4]:
brands_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1167 entries, 0 to 1166
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   _id           1167 non-null   object 
 1   barcode       1167 non-null   int64  
 2   category      1012 non-null   object 
 3   categoryCode  517 non-null    object 
 4   cpg           1167 non-null   object 
 5   name          1167 non-null   object 
 6   topBrand      555 non-null    float64
 7   brandCode     933 non-null    object 
dtypes: float64(1), int64(1), object(6)
memory usage: 73.1+ KB


In [5]:
users_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 495 entries, 0 to 494
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   _id           495 non-null    object
 1   active        495 non-null    bool  
 2   createdDate   495 non-null    object
 3   lastLogin     433 non-null    object
 4   role          495 non-null    object
 5   signUpSource  447 non-null    object
 6   state         439 non-null    object
dtypes: bool(1), object(6)
memory usage: 23.8+ KB


## Cleaning Receipts data

In [6]:
receipts_df.head()

,_id,bonusPointsEarned,bonusPointsEarnedReason,createDate,dateScanned,finishedDate,modifyDate,pointsAwardedDate,pointsEarned,purchaseDate,purchasedItemCount,rewardsReceiptItemList,rewardsReceiptStatus,totalSpent,userId
0,{'$oid': '5ff1e1eb0a720f0523000575'},500.0,"Receipt number 2 completed, bonus point schedu...",{'$date': 1609687531000},{'$date': 1609687531000},{'$date': 1609687531000},{'$date': 1609687536000},{'$date': 1609687531000},500.0,{'$date': 1609632000000},5.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,26.0,5ff1e1eacfcf6c399c274ae6
1,{'$oid': '5ff1e1bb0a720f052300056b'},150.0,"Receipt number 5 completed, bonus point schedu...",{'$date': 1609687483000},{'$date': 1609687483000},{'$date': 1609687483000},{'$date': 1609687488000},{'$date': 1609687483000},150.0,{'$date': 1609601083000},2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,11.0,5ff1e194b6a9d73a3a9f1052
2,{'$oid': '5ff1e1f10a720f052300057a'},5.0,All-receipts receipt bonus,{'$date': 1609687537000},{'$date': 1609687537000},NaN,{'$date': 1609687542000},NaN,5.0,{'$date': 1609632000000},1.0,"[{'needsFetchReview': False, 'partnerItemId': ...",REJECTED,10.0,5ff1e1f1cfcf6c399c274b0b
3,{'$oid': '5ff1e1ee0a7214ada100056f'},5.0,All-receipts receipt bonus,{'$date': 1609687534000},{'$date': 1609687534000},{'$date': 1609687534000},{'$date': 1609687539000},{'$date': 1609687534000},5.0,{'$date': 1609632000000},4.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,28.0,5ff1e1eacfcf6c399c274ae6
4,{'$oid': '5ff1e1d20a7214ada1000561'},5.0,All-receipts receipt bonus,{'$date': 1609687506000},{'$date': 1609687506000},{'$date': 1609687511000},{'$date': 1609687511000},{'$date': 1609687506000},5.0,{'$date': 1609601106000},2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,1.0,5ff1e194b6a9d73a3a9f1052


In [7]:
# Adjust column names, normalize fields, and convert dates to datetime
receipts_df['_id'] = pd.json_normalize(receipts_df['_id'])

receipts_df['createDate'] = pd.to_datetime(pd.json_normalize(receipts_df['createDate'])['$date'], unit = 'ms').astype('datetime64[s]')
receipts_df['dateScanned'] = pd.to_datetime(pd.json_normalize(receipts_df['dateScanned'])['$date'], unit = 'ms').astype('datetime64[s]')
receipts_df['finishedDate'] = pd.to_datetime(pd.json_normalize(receipts_df['finishedDate'])['$date'], errors = 'coerce',unit = 'ms').astype('datetime64[s]')
receipts_df['modifyDate'] = pd.to_datetime(pd.json_normalize(receipts_df['modifyDate'])['$date'], unit = 'ms').astype('datetime64[s]')
receipts_df['pointsAwardedDate'] = pd.to_datetime(pd.json_normalize(receipts_df['pointsAwardedDate'])['$date'], unit = 'ms').astype('datetime64[s]')
receipts_df['purchaseDate'] = pd.to_datetime(pd.json_normalize(receipts_df['purchaseDate'])['$date'], unit = 'ms').astype('datetime64[s]')

receipts_df = receipts_df.rename(columns={'_id': 'receiptId'})

In [8]:
# Explode on rewardsReceiptItemList
receipts_temp_df = receipts_df.explode('rewardsReceiptItemList')
receipts_temp_df.reset_index(inplace=True)

In [9]:
# Fill NAs for 'rewardsReceiptItemList' field
receipts_temp_df['rewardsReceiptItemList'] = receipts_temp_df['rewardsReceiptItemList'].fillna({i : {} for i in receipts_df.index})

In [10]:
# Breakout dict column and merge with original df
rewards_df = pd.json_normalize(receipts_temp_df['rewardsReceiptItemList'])
# rewards_df = rewards_df.add_prefix('rewardsReceiptItemList.')

rewards_df = pd.merge(receipts_temp_df[['receiptId', 'userId']], rewards_df, left_index = True, right_index = True, how = 'outer')
receipts_df.drop(columns = ['rewardsReceiptItemList'], axis = 1, inplace = True) # Drop dict col

In [11]:
rewards_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7381 entries, 0 to 7380
Data columns (total 36 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   receiptId                           7381 non-null   object 
 1   userId                              7381 non-null   object 
 2   barcode                             3090 non-null   object 
 3   description                         6560 non-null   object 
 4   finalPrice                          6767 non-null   object 
 5   itemPrice                           6767 non-null   object 
 6   needsFetchReview                    813 non-null    object 
 7   partnerItemId                       6941 non-null   object 
 8   preventTargetGapPoints              358 non-null    object 
 9   quantityPurchased                   6767 non-null   float64
 10  userFlaggedBarcode                  337 non-null    object 
 11  userFlaggedNewItem                  323 non

## Cleaning Users data

In [12]:
users_df.head()

,_id,active,createdDate,lastLogin,role,signUpSource,state
0,{'$oid': '5ff1e194b6a9d73a3a9f1052'},True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI
1,{'$oid': '5ff1e194b6a9d73a3a9f1052'},True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI
2,{'$oid': '5ff1e194b6a9d73a3a9f1052'},True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI
3,{'$oid': '5ff1e1eacfcf6c399c274ae6'},True,{'$date': 1609687530554},{'$date': 1609687530597},consumer,Email,WI
4,{'$oid': '5ff1e194b6a9d73a3a9f1052'},True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI


In [13]:
# Adjust column names and convert dates to datetime
users_df['_id'] = pd.json_normalize(users_df['_id'])

users_df['createdDate'] = pd.to_datetime(pd.json_normalize(users_df['createdDate'])['$date'], unit = 'ms').astype('datetime64[s]')
users_df['lastLogin'] = pd.to_datetime(pd.json_normalize(users_df['lastLogin'])['$date'], unit = 'ms').astype('datetime64[s]')

users_df = users_df.rename(columns={'_id': 'userId'})

## Cleaning Brands data

In [14]:
brands_df.head()

,_id,barcode,category,categoryCode,cpg,name,topBrand,brandCode
0,{'$oid': '601ac115be37ce2ead437551'},511111019862,Baking,BAKING,"{'$id': {'$oid': '601ac114be37ce2ead437550'}, ...",test brand @1612366101024,0.0,NaN
1,{'$oid': '601c5460be37ce2ead43755f'},511111519928,Beverages,BEVERAGES,"{'$id': {'$oid': '5332f5fbe4b03c9a25efd0ba'}, ...",Starbucks,0.0,STARBUCKS
2,{'$oid': '601ac142be37ce2ead43755d'},511111819905,Baking,BAKING,"{'$id': {'$oid': '601ac142be37ce2ead437559'}, ...",test brand @1612366146176,0.0,TEST BRANDCODE @1612366146176
3,{'$oid': '601ac142be37ce2ead43755a'},511111519874,Baking,BAKING,"{'$id': {'$oid': '601ac142be37ce2ead437559'}, ...",test brand @1612366146051,0.0,TEST BRANDCODE @1612366146051
4,{'$oid': '601ac142be37ce2ead43755e'},511111319917,Candy & Sweets,CANDY_AND_SWEETS,"{'$id': {'$oid': '5332fa12e4b03c9a25efd1e7'}, ...",test brand @1612366146827,0.0,TEST BRANDCODE @1612366146827


In [15]:
# Adjust column names
brands_df['_id'] = pd.json_normalize(brands_df['_id'])

brands_df = brands_df.rename(columns={'_id': 'brandId'})

In [16]:
# Breakout dict column and merge with original df
cpg_df = pd.json_normalize(brands_df['cpg'])
cpg_df = cpg_df.add_prefix('cpg.')

brands_merged_df = pd.merge(brands_df, cpg_df, left_index = True, right_index = True, how = 'outer')
brands_merged_df.drop(columns = ['cpg'],axis = 1, inplace = True) # Drop dict col

In [17]:
brands_merged_df.head()

,brandId,barcode,category,categoryCode,name,topBrand,brandCode,cpg.$ref,cpg.$id.$oid
0,601ac115be37ce2ead437551,511111019862,Baking,BAKING,test brand @1612366101024,0.0,NaN,Cogs,601ac114be37ce2ead437550
1,601c5460be37ce2ead43755f,511111519928,Beverages,BEVERAGES,Starbucks,0.0,STARBUCKS,Cogs,5332f5fbe4b03c9a25efd0ba
2,601ac142be37ce2ead43755d,511111819905,Baking,BAKING,test brand @1612366146176,0.0,TEST BRANDCODE @1612366146176,Cogs,601ac142be37ce2ead437559
3,601ac142be37ce2ead43755a,511111519874,Baking,BAKING,test brand @1612366146051,0.0,TEST BRANDCODE @1612366146051,Cogs,601ac142be37ce2ead437559
4,601ac142be37ce2ead43755e,511111319917,Candy & Sweets,CANDY_AND_SWEETS,test brand @1612366146827,0.0,TEST BRANDCODE @1612366146827,Cogs,5332fa12e4b03c9a25efd1e7


## Load into SQLite DB

In [18]:
# Create SQL DB and load data
fetch_sql = create_engine('sqlite://', echo = False)

users_df.drop_duplicates(subset=['userId']).to_sql('users', con = fetch_sql)
receipts_df.to_sql('receipts', con = fetch_sql)
rewards_df.to_sql('rewards', con = fetch_sql)
brands_merged_df.to_sql('brands',con = fetch_sql)

1167

# SQL Queries (SQLite)

For the queries, there were a lot of mismatches between what the query was asking for and the data available (e.g. mismatching brandId, no data in date range, no 'ACCEPTED' receipt status). For the purpose of the excercise however, I decided to write the SQL queries to be in line with what was being asked even if the results were blank or flawed because of these issues.

* What are the top 5 brands by receipts scanned for most recent month?

In [19]:
with fetch_sql.connect() as connection:
    result = connection.execute(text(
        """
        SELECT b.name from receipts r
        JOIN rewards rw ON r.receiptId = rw.receiptId
        JOIN brands b ON rw.barcode = b.barcode
        WHERE strftime('%Y-%m', r.dateScanned) = (SELECT max(strftime('%Y-%m', dateScanned)) from receipts)
        GROUP BY b.name
        ORDER BY count(*) DESC
        LIMIT 5;
        """))

    output = result.fetchall()

output

[]

* How does the ranking of the top 5 brands by receipts scanned for the recent month compare to the ranking for the previous month?

In [20]:
with fetch_sql.connect() as connection:
    result = connection.execute(text(
        """
        SELECT b.name from receipts r
        JOIN rewards rw ON r.receiptId = rw.receiptId
        JOIN brands b ON rw.barcode = b.barcode
        WHERE strftime('%Y-%m', r.dateScanned) = (SELECT strftime('%Y-%m', DATE(MAX(dateScanned), '-1 month')) from receipts)
        GROUP BY b.name
        ORDER BY count(*) DESC
        LIMIT 5;
        """))

    output = result.fetchall()

output

[]

* When considering average spend from receipts with 'rewardsReceiptStatus’ of ‘Accepted’ or ‘Rejected’, which is greater?

* When considering total number of items purchased from receipts with 'rewardsReceiptStatus’ of ‘Accepted’ or ‘Rejected’, which is greater?

In [21]:
with fetch_sql.connect() as connection:
    result = connection.execute(text(
        """
        SELECT rewardsReceiptStatus, AVG(totalSpent) as avgSpent, sum(purchasedItemCount) as itemsPurchased from receipts r
        JOIN rewards rw ON r.receiptId = rw.receiptId
        WHERE rewardsReceiptStatus in ('ACCEPTED', 'REJECTED')
        GROUP BY rewardsReceiptStatus;
        """))

    output = result.fetchall()

output

[('REJECTED', 19.544970059880253, 740.0)]

* Which brand has the most spend among users who were created within the past 6 months?

* Which brand has the most transactions among users who were created within the past 6 months?

In [22]:
with fetch_sql.connect() as connection:
    result = connection.execute(text(
        """
        SELECT
          (
          SELECT b.name as topSpender from rewards rw
          JOIN brands b ON rw.barcode = b.barcode
          JOIN users u ON rw.userId = u.userId
          WHERE u.createdDate >= DATE('now', '-6 months')
          GROUP BY b.name
          ORDER BY SUM(finalPrice) desc
          LIMIT 1
          ) as topSpender,
          (
          SELECT b.name as topSpender from rewards rw
          JOIN brands b ON rw.barcode = b.barcode
          JOIN users u ON rw.userId = u.userId
          WHERE u.createdDate >= DATE('now', '-6 months')
          GROUP BY b.name
          ORDER BY count(name) desc
          LIMIT 1
          ) as topTransactions;
        """))

    output = result.fetchall()

output

[(None, None)]

## Data Quality Checks

### Users

In [23]:
# Duplicate records in Users
users_df[users_df.duplicated()].count()

,0
userId,283
active,283
createdDate,283
lastLogin,261
role,283
signUpSource,240
state,233


In [24]:
# Some NULL values, doesn't seem like these fields would necessarily be cause for concern
users_df.isna().sum() / len(users_df) * 100

,0
userId,0.000000
active,0.000000
createdDate,0.000000
lastLogin,12.525253
role,0.000000
signUpSource,9.696970
state,11.313131


In [25]:
# Should only be 'CONSUMER'
users_df['role'].unique()

array(['consumer', 'fetch-staff'], dtype=object)

In [26]:
# Check that createdDate is before lastLogin
users_df[users_df['lastLogin'] < users_df['createdDate']]

,userId,active,createdDate,lastLogin,role,signUpSource,state


In [27]:
# Make sure all state values are abbreviated
users_df[users_df['state'].str.len() > 2]

,userId,active,createdDate,lastLogin,role,signUpSource,state


In [28]:
# Make sure values are boolean
users_df['active'].unique()

array([ True, False])

### Brands

In [29]:
# No duplicate records for brands
brands_merged_df[brands_merged_df.duplicated()].count()

,0
brandId,0
barcode,0
category,0
categoryCode,0
name,0
topBrand,0
brandCode,0
cpg.$ref,0
cpg.$id.$oid,0


In [30]:
# High percentage of NULL values in some features that might be needed
brands_merged_df.isna().sum() / len(brands_merged_df) * 100

,0
brandId,0.000000
barcode,0.000000
category,13.281919
categoryCode,55.698372
name,0.000000
topBrand,52.442159
brandCode,20.051414
cpg.$ref,0.000000
cpg.$id.$oid,0.000000


In [31]:
# Make sure values are boolean
brands_merged_df['topBrand'].unique()

array([ 0., nan,  1.])

### Receipts

In [32]:
# No duplicate records for receipts
receipts_df[receipts_df.duplicated()].count()

,0
receiptId,0
bonusPointsEarned,0
bonusPointsEarnedReason,0
createDate,0
dateScanned,0
finishedDate,0
modifyDate,0
pointsAwardedDate,0
pointsEarned,0
purchaseDate,0


In [33]:
# Some high NULL %s but could be from lack of purchases
receipts_df.isna().sum() / len(receipts_df) * 100

,0
receiptId,0.000000
bonusPointsEarned,51.385165
bonusPointsEarnedReason,51.385165
createDate,0.000000
dateScanned,0.000000
finishedDate,49.240393
modifyDate,0.000000
pointsAwardedDate,52.010724
pointsEarned,45.576408
purchaseDate,40.035746


### Rewards

In [34]:
# No duplicate records for rewards
rewards_df[rewards_df.duplicated()].count()

,0
receiptId,0
userId,0
barcode,0
description,0
finalPrice,0
itemPrice,0
needsFetchReview,0
partnerItemId,0
preventTargetGapPoints,0
quantityPurchased,0


In [35]:
# Lots of NULLS here, high enough that some of these columns might not be worth keeping
# Most concerning is 'barcode' which we're using as a FK
rewards_df.isna().sum() / len(rewards_df) * 100

,0
receiptId,0.000000
userId,0.000000
barcode,58.135754
description,11.123154
finalPrice,8.318656
itemPrice,8.318656
needsFetchReview,88.985232
partnerItemId,5.961252
preventTargetGapPoints,95.149709
quantityPurchased,8.318656


In [36]:
# There are a fair number of userIds in rewards_df but not users_df
# Could be due to test data but I'd want to know why these users aren't registered
rewards_df.loc[~rewards_df['userId'].isin(users_df['userId']), 'userId'].nunique()

117